In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageChops

In [2]:
from keras import applications
from keras.models import Model,load_model
from keras.layers import Input,concatenate

C:\Users\hp\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
model = applications.vgg19.VGG19(weights=None, include_top=False, pooling='max')
for layer in model.layers[:15]:
    layer.trainable = False

In [4]:
img_width, img_height, channels = 224, 224, 3
dim = (img_width, img_height)
def_thresh=38

In [5]:
anchor_in = Input(shape=(img_width, img_height, channels))
pos_in = Input(shape=(img_width, img_height, channels))
neg_in = Input(shape=(img_width, img_height, channels))

anchor_out = model(anchor_in)
pos_out = model(pos_in)
neg_out = model(neg_in)
merged_vector = concatenate([anchor_out, pos_out, neg_out],axis=1)

model = Model(inputs=[anchor_in, pos_in, neg_in], outputs=merged_vector)

In [6]:
model.load_weights("D:/Completed projects/Axis bank signatures/models/model_weights.h5")

In [7]:
def trim(im):
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    if bbox:
        return im.crop(bbox)

def to_rgb(img):
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 
    img_rgb = np.asarray(np.dstack((img, img, img)), dtype=np.uint8)
    return img_rgb

def returnimages(path, thr=True, crop=True):
    image=cv2.imread(path)                  #bringing the image
    #image=cv2.resize(image, (img_width, img_height))
    image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)    #converting to grey scale
    if thr==True:
        retval2,image = cv2.threshold(image,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)   #using OTSU thersholding
    if crop==True:
        threshold=to_rgb(image)
        image = trim(Image.fromarray(threshold))
        image = np.asarray(image)
        image=cv2.copyMakeBorder(image,3,3,10,10,cv2.BORDER_CONSTANT,value=[255,255,255])

    image=cv2.resize(image, (img_width, img_height))
    image=image.reshape(1,img_width, img_height,3)/255.0       #resizing and normalizing    
    return image

In [8]:
gen="D:/Completed projects/Axis bank signatures/sample_Signature/genuine/NFI-00101001.png"
forg="D:/Completed projects/Axis bank signatures/sample_Signature/forged/NFI-00301001.png"

In [9]:
img_gen=returnimages(gen, thr=True, crop=True)
img_forg=returnimages(forg, thr=True, crop=True)

In [10]:
x=model.predict([img_gen,img_forg,img_forg])
a1, a2, a3 = x[0,0:512], x[0,512:1024], x[0,1024:1536]

In [11]:
dist=np.linalg.norm(a1-a2)
if(dist>38):
    print("Forgery")
else:
    print("Genuine")
print(dist)

Forgery
38.508728
